**References:**

https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/

https://theailearner.com/tag/baseline-early-stopping-keras/

https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint

In [1]:
from tensorflow.keras import models, layers
from tensorflow.keras.datasets import mnist
import numpy as np

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [3]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images= train_images.astype('float32') / 255 # rescale pixel values from range [0, 255] to [0, 1]

test_images = test_images.reshape((10000, 28, 28, 1))
test_images= test_images.astype('float32') / 255

validation_images = train_images[50000:]
validation_labels = train_labels[50000:]

train_images = train_images[:50000]
train_labels = train_labels[:50000]

In [4]:
def make_module():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(10, activation='softmax'))

  model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

In [5]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor="val_accuracy",mode="max",min_delta=0.001,patience=3,restore_best_weights=True,verbose=1)#,baseline=0.99)
# monitor = loss,accuracy,val_loss,val_accuracy
# mode = min(minimize),max(maximize),auto(loss=minimize(min),accuracy=maximize(max))
# patience = nombre d'epoche qui va attend pour verifié
# min_delta = used for val_accuracy - min_delta
# baseline = The training process stops because of the val_acc – min_delta < baseline

In [6]:
model = make_module()
history = model.fit(train_images, train_labels, epochs=25, batch_size=64, validation_data=(validation_images, validation_labels),callbacks=[es])

Epoch 1/25
782/782 [==============================] - 7s 9ms/step - loss: 0.2010 - accuracy: 0.9371 - val_loss: 0.0645 - val_accuracy: 0.9821
Epoch 2/25
782/782 [==============================] - 6s 8ms/step - loss: 0.0559 - accuracy: 0.9821 - val_loss: 0.0430 - val_accuracy: 0.9857
Epoch 3/25
782/782 [==============================] - 6s 8ms/step - loss: 0.0385 - accuracy: 0.9883 - val_loss: 0.0425 - val_accuracy: 0.9874
Epoch 4/25
782/782 [==============================] - 6s 8ms/step - loss: 0.0279 - accuracy: 0.9915 - val_loss: 0.0379 - val_accuracy: 0.9884
Epoch 5/25
782/782 [==============================] - 6s 8ms/step - loss: 0.0217 - accuracy: 0.9930 - val_loss: 0.0427 - val_accuracy: 0.9898
Epoch 6/25
782/782 [==============================] - 6s 8ms/step - loss: 0.0179 - accuracy: 0.9943 - val_loss: 0.0335 - val_accuracy: 0.9909
Epoch 7/25
782/782 [==============================] - 6s 8ms/step - loss: 0.0141 - accuracy: 0.9955 - val_loss: 0.0363 - val_accuracy: 0.9907
Epoch 

In [7]:
model.evaluate(validation_images, validation_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.0335 - accuracy: 0.9909


[0.03347000479698181, 0.9908999800682068]

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True,verbose=1)

In [9]:
model = make_module()
history = model.fit(train_images, train_labels, epochs=25, batch_size=64, validation_data=(validation_images, validation_labels),callbacks=[mc])

Epoch 1/25
780/782 [============================>.] - ETA: 0s - loss: 0.1976 - accuracy: 0.9381
Epoch 00001: val_loss improved from inf to 0.06359, saving model to best_model.h5
782/782 [==============================] - 7s 9ms/step - loss: 0.1974 - accuracy: 0.9382 - val_loss: 0.0636 - val_accuracy: 0.9821
Epoch 2/25
780/782 [============================>.] - ETA: 0s - loss: 0.0532 - accuracy: 0.9836
Epoch 00002: val_loss improved from 0.06359 to 0.04961, saving model to best_model.h5
782/782 [==============================] - 7s 9ms/step - loss: 0.0532 - accuracy: 0.9836 - val_loss: 0.0496 - val_accuracy: 0.9848
Epoch 3/25
779/782 [============================>.] - ETA: 0s - loss: 0.0365 - accuracy: 0.9887
Epoch 00003: val_loss improved from 0.04961 to 0.03849, saving model to best_model.h5
782/782 [==============================] - 7s 9ms/step - loss: 0.0365 - accuracy: 0.9887 - val_loss: 0.0385 - val_accuracy: 0.9889
Epoch 4/25
777/782 [============================>.] - ETA: 0s - l

In [10]:
model.evaluate(validation_images, validation_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.1010 - accuracy: 0.9910


[0.10099881887435913, 0.9909999966621399]

In [12]:
from tensorflow.keras.models import load_model
model = load_model("best_model.h5")
model.evaluate(validation_images, validation_labels)

313/313 [==============================] - 1s 4ms/step - loss: 0.0374 - accuracy: 0.9911


[0.03740204870700836, 0.991100013256073]